In [ ]:
from typing import Annotated, Literal, TypedDict
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from pinecone import Pinecone, ServerlessSpec

from difflib import get_close_matches
import json
import os
from dotenv import load_dotenv
load_dotenv()

True

In [86]:
INDEX_NAME = "funeral-services"
pc = Pinecone()
index = pc.Index(INDEX_NAME)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
print(index.describe_index_stats(), '\n\n')

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'funeral_facilities': {'vector_count': 2518},
                'guide': {'vector_count': 4},
                'ordinance': {'vector_count': 234}},
 'total_vector_count': 2756,
 'vector_type': 'dense'} 




In [87]:
vectorstore_ordinance = PineconeVectorStore(
    index=index, 
    embedding=embeddings,
    namespace="ordinance"
)
vectorstore_funeral_facilities = PineconeVectorStore(
    index=index, 
    embedding=embeddings,
    namespace='funeral_facilities'
)

In [85]:
with open('../data/processed/region_list.json', 'r', encoding='utf-8') as f:
    region_list_json = json.load(f)
    print(region_list_json)

{'public_funeral_ordinance': ['가평군', '강화군', '경기도', '고양시', '과천시', '광명시', '광주시', '구리시', '김포시', '남동구', '남양주시', '동두천시', '부천시', '서울특별시', '서울특별시 강남구', '서울특별시 강동구', '서울특별시 강북구', '서울특별시 강서구', '서울특별시 관악구', '서울특별시 광진구', '서울특별시 구로구', '서울특별시 금천구', '서울특별시 노원구', '서울특별시 도봉구', '서울특별시 동대문구', '서울특별시 동작구', '서울특별시 마포구', '서울특별시 서대문구', '서울특별시 서초구', '서울특별시 성동구', '서울특별시 성북구', '서울특별시 양천구', '서울특별시 영등포구', '서울특별시 용산구', '서울특별시 은평구', '서울특별시 종로구', '서울특별시 중구', '서울특별시 중랑구', '성남시', '수원시', '시흥시', '안산시', '안성시', '안양시', '양주시', '양평군', '여주시', '연천군', '오산시', '옹진군', '용인시', '의왕시', '의정부시', '이천시', '인천광역시', '인천광역시 계양구', '인천광역시 동구', '인천광역시 미추홀구', '인천광역시 부평구', '인천광역시 서구', '인천광역시 연수구', '인천광역시 중구', '파주시', '포천시', '화성시'], 'cremation_words_support': [], 'cremation_detail': ['강원도 고성군', '강원도 삼척시', '강원도 양구군', '강원도 양양군', '강원도 영월군', '강원도 철원군', '강원도 평창군', '강원도 화천군', '경기도 가평군', '경기도 과천시', '경기도 광주시', '경기도 구리시', '경기도 김포시', '경기도 남양주시', '경기도 안성시', '경기도 양주시', '경기도 양평군', '경기도 연천군', '경기도 오산시', '경기도 의왕시', '경기도 이천시', '경기도 평택시', '경기도 하남시', '경남 거창군', '경남 산

In [84]:
with open('../data/processed/facilities_region_list.json', 'r', encoding='utf-8') as f:
    facilities_region_list_json = json.load(f)
    print(facilities_region_list_json)

{'cemetery': ['강원도 태백시', '강원특별자치도 강릉시', '강원특별자치도 고성군', '강원특별자치도 동해시', '강원특별자치도 삼척시', '강원특별자치도 속초시', '강원특별자치도 양구군', '강원특별자치도 양양군', '강원특별자치도 영월군', '강원특별자치도 원주시', '강원특별자치도 인제군', '강원특별자치도 정선군', '강원특별자치도 철원군', '강원특별자치도 춘천시', '강원특별자치도 평창군', '강원특별자치도 홍천군', '강원특별자치도 화천군', '강원특별자치도 횡성군', '경기 파주시', '경기도 가평군', '경기도 고양시', '경기도 광주시', '경기도 구리시', '경기도 김포시', '경기도 남양주시', '경기도 동두천시', '경기도 성남시', '경기도 시흥시', '경기도 안산시', '경기도 안성시', '경기도 양주시', '경기도 양평군', '경기도 여주시', '경기도 연천군', '경기도 오산시', '경기도 용인시', '경기도 의왕시', '경기도 의정부시', '경기도 이천시', '경기도 파주시', '경기도 평택시', '경기도 포천시', '경기도 화성시', '경상남도 거제시', '경상남도 거창군', '경상남도 고성군', '경상남도 김해시', '경상남도 남해군', '경상남도 사천시', '경상남도 양산시', '경상남도 의령군', '경상남도 진주시', '경상남도 창원시', '경상남도 함안군', '경상남도 함양군', '경상북도 경산시', '경상북도 경주시', '경상북도 구미시', '경상북도 김천시', '경상북도 상주시', '경상북도 성주군', '경상북도 안동시', '경상북도 영덕군', '경상북도 울진군', '경상북도 칠곡군', '경상북도 포항시', '광주광역시 남구', '광주광역시 북구', '대구광역시 군위군', '대구광역시 달서구', '대구광역시 달성군', '대구광역시 북구', '대구광역시 수성구', '대구광역시 중구', '대전광역시 동구', '대전광역시 서구', '부산광역시 금정구', '부산광역시 기장군', '부산광역시 남구', '서울특별

In [67]:
def find_matching_region(user_input, region_list):
    # 1. 양방향 체크
    for region in region_list:
        if user_input in region or region in user_input:
            return region
    
    # 2. 유사도 기반 매칭
    matches = get_close_matches(user_input, region_list, n=1, cutoff=0.6)
    return matches[0] if matches else None

In [68]:
def find_matching_regions(user_input, region_list, n=3):
    """유사한 지역 여러 개 반환"""
    matched = []
    
    # 1. 양방향 체크
    for region in region_list:
        if user_input in region or region in user_input:
            matched.append(region)
            if len(matched) >= n:
                return matched
    
    # 2. 유사도 기반 매칭
    if not matched:
        matched = get_close_matches(user_input, region_list, n=n, cutoff=0.6)
    
    return matched if matched else None

In [69]:
find_matching_regions("서울", region_list = region_list_json["public_funeral_ordinance"])

['서울특별시', '서울특별시 강남구', '서울특별시 강동구']

In [88]:
@tool
def search_public_funeral_ordinance(query: str, region: str = None):
    """
    공영장례 조례를 검색합니다.
    
    Args:
        query: 검색어 (예: "지원 대상")
        region: 지역명 (예: "수원시", "서울특별시 강남구, 인천광역시 서구")
    """
    # filter_dict 먼저 초기화
    filter_dict = {"type": "Public_Funeral_Ordinance"}
    k = 3
    
    if region:
        region_list = region_list_json["public_funeral_ordinance"]
        matched = find_matching_regions(region, region_list, n=k)  # 여러 개
        print(f"공영 장례 조례 매칭된 지역들 최대 {k}개",matched)
        if matched:
            if len(matched) == 1:
                filter_dict["region"] = matched[0]  # 1개면 직접
            else:
                filter_dict["region"] = {"$in": matched}  # 여러 개면 in

    results = vectorstore_ordinance.similarity_search(query, k=k, filter=filter_dict)
    # print(f"공영 장례 조례 검색된 문서 개수: {len(results)}")  # 추가
    # for i, doc in enumerate(results):
    #     print(f"{i+1}. {doc.metadata.get('region')}")  # 지역 확인
    print("툴 검색 결과:",results)
    return results

In [89]:
@tool
def search_cremation_subsidy_ordinance(query: str, region: str = None):
    """
    화장 장려금 조례를 검색합니다.  
    제외 대상에 대한 정보가 말이 뒤죽박죽 되어 이해하기 어려울 경우 다음의 사항을 바탕으로 이해한다.
    1.「장사 등에 관한 법률」 제7조 제2항을 위반한 경우
    2. 다른 법령에 따라 화장에 대한 지원금을 받은 경우
    Args:
        query: 검색어 (예: "지원 대상")
        region: 지역명 (예: "강원도 고성군", "서울 강남")
    """
    filter_dict = {"type": "Cremation_Subsidy_Ordinance"}
    k = 3
    
    if region:
        region_list = region_list_json["cremation_detail"] + region_list_json["cremation_etcetera"]
        matched = find_matching_regions(region, region_list, n=k)  # 여러 개
        print(f"화장 장려금 조례 매칭된 지역들 최대 {k}개",matched)

        if matched:
            if len(matched) == 1:
                filter_dict["region"] = matched[0]  # 1개면 직접
            else:
                filter_dict["region"] = {"$in": matched}  # 여러 개면 in

    results = vectorstore_ordinance.similarity_search(query, k=k, filter=filter_dict)
    
    print("툴 검색 결과:",results)
    return results

In [90]:
import traceback # 상세 에러 확인을 위해 추가

@tool
def search_funeral_facilities(query: str, region: str = None):
    """
    장례 시설을 검색합니다. 묘지, 봉안당, 화장장, 자연장지, 장례식장을 모두 검색합니다.
    
    Args:
        query: 검색 쿼리 (예: "봉안당 찾아줘", "화장장 어디 있어?", "자연장지 추천")
        region: 지역명 (예: "서울", "경기도 수원시")
    """
    filter_dict = {}
    
    if region:
        # 모든 시설 타입의 지역 통합
        all_regions = []
        for regions in facilities_region_list_json.values():
            all_regions.extend(regions)
        all_regions = sorted(set(all_regions))
        
        matched = find_matching_regions(region, all_regions, n=3)
        if matched:
            if len(matched) == 1:
                filter_dict["full_region"] = matched[0]
            else:
                filter_dict["full_region"] = {"$in": matched}
    
    try:
        results = vectorstore_funeral_facilities.similarity_search(
            query, 
            k=5,
            filter=filter_dict
        )
        return results
    except Exception as e:
        return f"검색 중 오류: {str(e)}"

In [ ]:
# --- LangGraph 상태 및 노드 정의
tools = [search_public_funeral_ordinance, 
         search_cremation_subsidy_ordinance, 
         search_funeral_facilities]

# LLM 모델 로드 및 도구 바인딩
llm = ChatOpenAI(model="gpt-4o", temperature=0.1)
llm_with_tools = llm.bind_tools(tools)

# 그래프의 State 정의 (대화 기록 유지)
class AgentState(TypedDict):
    messages: Annotated[list, add_messages]

def agent_node(state: AgentState):
    # 현재까지의 대화 기록을 가져옴
    current_messages = state['messages']
    
    # 시스템 프롬프트 정의
    system_prompt = SystemMessage(content="""
    당신은 따뜻하고 사려 깊은 '죽기 전 필요한 정보를 전달하는 도우미 챗봇'입니다.
    사용자가 장례식장, 봉안당, 납골당 등의 정보를 묻거나, 죽음/임종/빈소/조문 등과 관련된 상황을 암시하면 
    반드시 tools 도구를 사용하여 적절한 내용을 찾아주세요.
    사용자가 구체적인 지역을 말하지 않았다면, 먼저 어느 지역을 찾는지 정중하게 물어보세요.

    tools로 검색 후 원하는 장소나 지역이 나오지 않을 경우 검색하신 곳에 대한 정보가 없음을 전달하고 
    유사한 정보를 가져왔음을 밝히고 답변해주세요.
    검색된 결과를 제공할 때는 위로의 말과 함께 정보를 정확하게 전달하세요.
    """)

    # 대화 기록 맨 앞에 시스템 메시지가 없으면, 이번 턴(invoke)에만 임시로 붙여서 보냄
    # (state에 영구 저장하지 않음으로써 중복 방지)
    if not isinstance(current_messages[0], SystemMessage):
        messages_to_send = [system_prompt] + current_messages
    else:
        messages_to_send = current_messages

    # LLM 호출
    response = llm_with_tools.invoke(messages_to_send)
    # print("response:", response)
    
    # 새로운 응답만 리스트에 담아 반환 (add_messages가 알아서 기존 기록 뒤에 붙여줌)
    return {"messages": [response]}

# [Node 2] 도구 실행 노드
tool_node = ToolNode(tools)

# [Edge] 조건부 엣지: 도구를 쓸지, 답변을 끝낼지 결정
def should_continue(state: AgentState) -> Literal["tools", "__end__"]:
    messages = state['messages']
    last_message = messages[-1]

    print(f"Tool Calls 존재 여부: {last_message.tool_calls}")
    
    # LLM이 도구 호출을 요청했는지 확인
    if last_message.tool_calls:
        return "tools"
    return "__end__"

In [79]:
# --- 그래프 구성 (Workflow)
workflow = StateGraph(AgentState)

# 노드 추가
workflow.add_node("agent", agent_node)
workflow.add_node("tools", tool_node)

# 엣지 연결
workflow.set_entry_point("agent") # 시작점
workflow.add_conditional_edges(
    "agent",
    should_continue,
)
workflow.add_edge("tools", "agent") # 도구 실행 후 다시 에이전트로 복귀 (결과 해석)

# 그래프 컴파일
app = workflow.compile()

In [80]:
# --- 실행 테스트
def chat(user_input):
    print(f"\n사용자: {user_input}")
    inputs = {"messages": [HumanMessage(content=user_input)]}
    
    # 그래프 실행 (스트리밍으로 과정 확인 가능)
    for event in app.stream(inputs):
        for key, value in event.items():
            if key == "agent":
                msg = value["messages"][0]
                # print(f"Agent 생각: {msg.content}") # 디버깅용
            elif key == "tools":
                # print(f"Tool 실행 결과...") # 디버깅용
                pass
    
    # 최종 응답 출력
    final_response = value["messages"][0].content
    print(f"챗봇: {final_response}")
    return final_response

In [83]:
rag_result = chat("대구 공영장례와 화장장려금 신청 대상에 대해 알려줘.")


사용자: 대구 공영장례와 화장장려금 신청 대상에 대해 알려줘.
Tool Calls 존재 여부: [{'name': 'search_public_funeral_ordinance', 'args': {'query': '지원 대상', 'region': '대구'}, 'id': 'call_q2wwXyTKbdzEvTPIJjXOJJNM', 'type': 'tool_call'}, {'name': 'search_cremation_subsidy_ordinance', 'args': {'query': '지원 대상', 'region': '대구'}, 'id': 'call_ewvUZM3HSurLGtmIPBfVPrFg', 'type': 'tool_call'}]
공영 장례 조례 매칭된 지역들 최대 3개 None
화장 장려금 조례 매칭된 지역들 최대 3개 ['대구광역시']
툴 검색 결과: [Document(id='a92ec158-29a4-4f44-80d8-ae7f5a8b5e76', metadata={'region': '인천광역시 서구', 'type': 'Public_Funeral_Ordinance'}, page_content='(현행, 제정) 2023.11.06. 제2080호\n인천광역시 서구\n무연고 사망자 등에 대한 장례지원 조례\n지원대상\n무연고 사망자 등에 대한 장례 지원대상은 서구 관내에서 사망한 사람으로 다음 어느 하나에 해당하는\n경우로 한다.\n1. 연고자 또는 부양의무자가 없거나, 연고자를 알 수 없는 경우\n2. 부양의무자가 미성년자 또는 장애 등으로 장례를 치를 능력이 없는 경우\n3. 부양의무자가 있으나 사회적ㆍ경제적ㆍ신체적 능력 부족 및 가족관계의 단절 등 불가피한 이유로\n시신 인수를 기피ㆍ거부하는 경우\n4. 그 밖에 구청장이 무연고 사망자 등 장례지원이 필요하다고 인정하는 경우\n지원내용\n구청장이 지원할 수 있는 내용은 다음과 같다.\n수의, 관, 염사, 그 밖에\n장례에 필요한 용품\n사체 검사비, 운반비, 영안실 안치료 포함\n장례업체ㆍ민간기관ㆍ\n비영리 단체

In [82]:
print(rag_result)

서울특별시의 공영장례와 화장장려금 지원 대상에 대한 정보를 아래와 같이 안내드립니다.

### 공영장례 지원 대상 (서울특별시)
서울특별시에서는 다음과 같은 경우에 공영장례를 지원할 수 있습니다:
1. 서울시 내에서 사망한 무연고 사망자 또는 연고자가 시신 인수를 거부하거나 기피한 경우.
2. 사망 시, 시에 주민등록을 두고 있는 자로서 장제급여 수급자 및 차상위 계층으로 연고자가 장례처리 능력이 없는 경우.
3. 사망 시, 시에 주민등록을 두고 있는 자로서 서울특별시 고독사 예방 및 사회적 고립 가구 안전망 확충을 위한 조례에 따른 고독사로 인정되는 경우.
4. 서울특별시 아동학대 예방 및 방지에 관한 조례에 따른 아동학대로 사망한 경우로 연고자가 장례를 치를 수 없는 경우.
5. 그 외 시청장 또는 구청장이 장례지원이 필요하다고 인정하는 경우.

### 화장장려금 지원 대상 (서울특별시)
서울특별시의 화장장려금 지원 대상에 대한 구체적인 정보는 제공되지 않았습니다. 다른 지역의 예시로는, 주민등록이 되어 있는 사람이나 외국인 등록이 되어 있는 경우, 그리고 특정 조건을 만족하는 경우에 지원이 가능할 수 있습니다. 서울특별시의 구체적인 지원 대상은 해당 구청이나 시청에 문의하시는 것이 좋습니다.

이러한 정보가 조금이나마 도움이 되길 바라며, 추가적인 질문이 있으시면 언제든지 말씀해 주세요.


In [ ]:
# 테스트 실행
user_query = "수원시 공영장례 지원 대상 알려줘"

# LLM이 tool 호출
inputs = {"messages": [HumanMessage(content=user_query)]}

for event in app.stream(inputs):
    for key, value in event.items():
        if key == "agent":
            msg = value["messages"][0]
            print(f"Agent: {msg.content}")
        elif key == "tools":
            print("Tool 실행 중...")

# 또는 직접 tool 테스트
print("\n=== Tool 직접 테스트 ===")
result = search_public_funeral_ordinance(
    query="지원 대상",
    region="수원"
)
print(f"검색 결과: {len(result)}건")
for i, doc in enumerate(result):
    print(f"\n[{i+1}] {doc.metadata.get('region')}")
    print(doc.page_content[:200])

# 지역 없이 테스트
print("\n=== 지역 필터 없이 ===")
result2 = search_public_funeral_ordinance(query="지원 대상")
print(f"검색 결과: {len(result2)}건")

In [8]:


# # 케이스 2: 암시적인 상황 (도구 호출 전 지역 확인 유도 예상)
# chat("갑자기 아버지가 돌아가셨어.. 어떡해야 할지 모르겠네.")

# # 케이스 3: 암시적 상황 + 지역 포함
# chat("지금 부산 해운대 쪽인데, 급하게 빈소를 알아봐야 해.")

In [9]:
# chat("부산광역시에 있는 봉안당을 알려줘")